In [1]:
import dotenv

dotenv.load_dotenv('../.env')

True

In [2]:
data = {
    'building_match': {'value': 'Warsaw Trade Tower'},  # Można zostawić jako default
    'office_match': {'value': '1'},  # Można zostawić jako default
    "preferred_city": {
        'description': 'Preferowane miasto',
        'title': 'Preferred City',
        'type': 'string',
        'value': 'Warszawa'
    },
    'preferred_district_or_area': {
        'description': 'Preferowana dzielnica, okolica lub konkretna ulica',
        'title': 'Preferred District Or Area',
        'type': 'string',
        'value': 'Wola / Centrum'
    },
    'preferred_floor': {
        'description': 'Preferowane piętro (np. parter, wysokie piętro, dowolne)',
        'title': 'Preferred Floor',
        'type': 'string',
        'value': "Dowolne (z dostępem do światła dziennego)"
    },
    'office_area_m2': {
        'description': 'Wymagana powierzchnia biura w m²',
        'exclusiveMinimum': 0,
        'title': 'Office Area M2',
        'type': 'integer',
        'value': 350  # Średnia z 300–400 m²
    },
    'number_of_employees': {
        'description': 'Liczba pracowników/stanowisk pracy',
        'exclusiveMinimum': 0,
        'title': 'Number Of Employees',
        'type': 'integer',
        'value': 30
    },
    'office_type': {
        'description': 'Rodzaj przestrzeni (np. Biuro prywatne, Open space, Coworking, Inne)',
        'title': 'Office Type',
        'type': 'string',
        'value': "Open space z wydzielonymi strefami do pracy i relaksu"
    },
    'rental_period_start': {
        'description': 'Data rozpoczęcia najmu (format RRRR-MM-DD)',
        'format': 'date',
        'title': 'Rental Period Start',
        'type': 'string',
        'value': "2025-06-01"
    },
    'rental_period_end': {
        'anyOf': [{'format': 'date', 'type': 'string'}, {'type': 'null'}],
        'default': None,
        'description': 'Data zakończenia najmu, jeśli dotyczy',
        'title': 'Rental Period End',
        'value': None  # Brak określonego końca
    },
    'rental_period_unlimited': {
        'description': 'Czy najem ma być na czas nieokreślony (true/false)',
        'title': 'Rental Period Unlimited',
        'type': 'boolean',
        'value': True
    },
    'access_hours': {
        'description': 'Godziny dostępu (np. 24/7, w godzinach pracy, inne)',
        'title': 'Access Hours',
        'type': 'string',
        'value': "24/7"
    },
    'monthly_budget_net_PLN': {
        'description': 'Miesięczny budżet netto w PLN',
        'exclusiveMinimum': 0,
        'title': 'Monthly Budget Net Pln',
        'type': 'number',
        'value': 25000.0  # Przybliżony budżet na 350 m²
    },
    "interior_style": {
        "description": "Preferowany styl aranżacji",
        "value": "Przytulny, z naturalnymi materiałami, dużą ilością zieleni i światła dziennego"
    },
    "layout_preferences": {
        "description": "Lista preferowanych układów biura",
        "value": [
            "Open space",
            "Chill-out room",
            "Strefy kreatywne / ciche pokoje do pracy"
        ]
    },
    "equipment_and_features": {
        "description": "Lista wymaganych elementów wyposażenia",
        "value": [
            "Krzesła ergonomiczne",
            "Biurka",
            "Konsola do gier",
            "Stół do ping-ponga lub piłkarzyki",
            "Wygodne pufy i sofy",
            "Rośliny doniczkowe",
            "Duże okna",
            "Stojaki na rowery",
            "Możliwość instalacji prysznica"
        ]
    },
    "additional_notes_on_design": {
        "description": "Dodatkowe wymagania i preferencje dotyczące aranżacji biura",
        "value": "Biuro powinno sprzyjać skupieniu i kreatywności, być wizualnie ciepłe, z elementami natury. Zielone przestrzenie w częściach wspólnych są kluczowe."
    }
}


In [3]:
inquiry_data = {k: v['value'] for k, v in data.items()}

In [4]:
import json
import openai

client = openai.Client()

def get_llm_summary(data):
    data_formatted = json.dumps(data, ensure_ascii=False, indent=2)
    
    messages = [
        {"role": "system", 
         "content": 
"""You are a commercial real estate assistant. Based on the following office inquiry data, write a structured summary in **Polish**. The summary must be formatted in 3 clear sections:
1. INFORMACJE O KLIENCIE - a bullet-point list including:
    - Miasto i dzielnica
    - Powierzchnia biura
    - Liczba pracowników
    - Typ przestrzeni
    - Preferowane piętro
    - Okres najmu (data od-do)
    - Czas nieokreślony (tak/nie)
    - Godziny dostępu
    - Budżet netto (PLN/miesiąc)

2. PREFERENCJE DOTYCZĄCE BUDYNKU - akapit w języku naturalnym opisujący:
    - Lokalizacja, dzielnica
    - Preferencje co do piętra i dostępu

3. PREFERENCJE DOTYCZĄCE BIURA I ARANŻACJI - akapit w języku naturalnym opisujący:
    - Styl wnętrza
    - Układ (open space, chill-out, etc.)
    - Wymagane wyposażenie
    - Dodatkowe uwagi o aranżacji
"""},
        {"role": "user", 
         "content": f"Preferencje użytkownika: \n{data_formatted}"}
    ]
        
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    
    return response.choices[0].message.content.strip()

In [5]:
response = get_llm_summary(inquiry_data)

response = response.replace('#', '').replace('*', '')

In [6]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import textwrap
import os

def export_summary_to_pdf_with_images(
    summary_text,
    image_paths,
    building_name="",
    floor_number="",
    filename="office_inquiry_summary.pdf"
):
    # Czcionka z polskimi znakami
    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
    pdfmetrics.registerFont(TTFont("DejaVuSans", font_path))

    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4
    margin_x = 2 * cm
    margin_y = 2 * cm
    usable_width = width - 2 * margin_x

    # === Title and subtitle ===
    title = "Zapytanie ofertowe dotyczące wynajmu biura"
    subtitle = f"Budynek: {building_name}, Piętro: {floor_number}"

    y_position = height - 2.5 * cm
    c.setFont("DejaVuSans", 18)
    c.drawCentredString(width / 2, y_position, title)

    y_position -= 1 * cm
    c.setFont("DejaVuSans", 13)
    c.drawCentredString(width / 2, y_position, subtitle)

    # === Summary below title ===
    textobject = c.beginText(margin_x, y_position - 2 * cm)
    textobject.setFont("DejaVuSans", 11)
    wrap_width = 80

    for line in summary_text.splitlines():
        if line.strip() == "":
            textobject.textLine("")
        else:
            for wrapped_line in textwrap.wrap(line, width=wrap_width):
                textobject.textLine(wrapped_line)

        if textobject.getY() < margin_y:
            c.drawText(textobject)
            c.showPage()
            textobject = c.beginText(margin_x, height - margin_y)
            textobject.setFont("DejaVuSans", 11)

    c.drawText(textobject)

    # === Images ===
    images_per_page = 2
    image_height = (height - 3 * margin_y) / images_per_page
    image_width = usable_width

    for i in range(0, len(image_paths), images_per_page):
        c.showPage()
        current_images = image_paths[i:i + images_per_page]

        for idx, img_path in enumerate(current_images):
            try:
                x = margin_x
                y = height - margin_y - (idx * (image_height + margin_y))
                c.drawImage(
                    img_path,
                    x,
                    y - image_height,
                    width=image_width,
                    height=image_height,
                    preserveAspectRatio=True,
                    anchor='n'
                )
            except Exception as e:
                print(f"❌ Błąd przy wstawianiu obrazu {img_path}: {e}")

    c.save()
    print(f"✅ PDF wygenerowany jako: {filename}")


In [11]:
office_images = [
    '../office_mocks/space1/empty_office_3_1_inpaint.png',
    '../office_mocks/space2/empty_office_3_2_inpaint.png',
    '../office_mocks/space3/empty_office_3_3_inpaint3.png'
]

In [12]:
export_summary_to_pdf_with_images(response, office_images, inquiry_data['building_match'], inquiry_data['office_match'])

✅ PDF wygenerowany jako: office_inquiry_summary.pdf
